In [1]:
import pandas as pd
import os
import math
import numpy as np
import pandas as pd
import matplotlib as plt
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.autograd as autograd

In [2]:
# set seed for experiment
def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    # making sure GPU runs are deterministic even if they are slower
    print("Seeded everything: {}".format(seed))

In [3]:
set_seed(42)

Seeded everything: 42


# Test how the straight through estimator works

$$\text{With indicators:}\quad f(x_1, x_2) = \mathbb{1}_{x_1 > 5}x_1^2 + \mathbb{1}_{x_2 > 5}x_2^2$$
$$\text{Without indicators:}\quad g(x_1, x_2) = x_1^2 + x_2^2$$

In [4]:
# returns 1_{x > 5}
class GetIndicators(autograd.Function):
    @staticmethod
    def forward(ctx, x):
        """
            In the forward pass we receive a Tensor containing the input and return
            a Tensor containing the output. ctx is a context object that can be used
            to stash information for backward computation. You can cache arbitrary
            objects for use in the backward pass using the ctx.save_for_backward method.
        """
        mask = (x > 5).int()
        return mask
    
    @staticmethod
    def backward(ctx, g):
        """
            In the backward pass we receive a Tensor containing the gradient of the loss
            with respect to the output, and we need to compute the gradient of the loss
            with respect to the input.
            NOTE: We need only "g" here because we have only one input -> x
        """
        # send the gradient g straight-through on the backward pass.
        print("Incoming grad = Outgoing grad = {}".format(g))
        return g

## See if the indicator function works as expected (it does)

In [5]:
x = torch.Tensor([1, 6])
GetIndicators.apply(x)

tensor([0, 1], dtype=torch.int32)

In [6]:
def f(x):
    mask = GetIndicators.apply(x)
    out = (mask * x** 2).sum()
    return out

def g(x):
    out = (x**2).sum()
    return out

$$\nabla g(x) = \begin{bmatrix} 2x_1\\ 2x_2 \end{bmatrix}$$

### Assuming STE
$$\frac{\partial f(x)}{\partial x_i} =  ?$$

### Now check if that is what we get

In [7]:
x = torch.Tensor([2, 6])
x.requires_grad=True
f_out = f(x)
f_out.backward()
print("grad(f(x)) = {}".format(x.grad.data))

grad(f(x)) = tensor([ 0., 12.])


In [8]:
x = torch.Tensor([2, 6])
x.requires_grad=True
g_out = g(x)
g_out.backward()
print("grad(g(x)) = {}".format(x.grad.data))

grad(g(x)) = tensor([ 4., 12.])


## This doesn't make sense. The gradient of g is as expected (2x) but the gradient of f seems to be $\frac{\partial f(x)}{\partial x_i} = \mathbb{1}_{x_i > 5} 2x_i$? I expected it to apply the chain rule and then use the STE.